## 필요한 라이브러리 Import

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

## 네이버증권 Financial Summary 크롤링

기본적으로 네이버증권은 URL에 종목코드륿 변수로 받아 종목코드별(기업별) 페이지를 구성하고 있다.<br><br>
Chrome 개발자 도구에 들어가 Network를 켜보면 "cF1001.aspx?cmp=cd"부분을 보면 매출액, 영업이익 등이 html 언어로 표현되어 있는 것을 알 수 있다. <br><br>
URL을 살펴보면 cmp_cd는 종목코드, freq_typ은 연간(A), 분기(Q), encparam과 id는 종목별로 랜덤하게 부여되는 key값 정도로 파악된다. <br><br>
encparam과 id는 랜덤 값으로 파악되기 때문에 이 정보를 알고 접근할 수는 없다. 그래서 2개의 값을 html에서 찾아서 접근하는 방법을 사용하기로 했다.

## encparam과 id 값 찾기

In [24]:
def get_basic_key_values(code):
    # 종목별 encparam, encid 찾기
    url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={code}'
    html = requests.get(url).text
    
    re_enc = re.compile("encparam: '(.*)'", re.IGNORECASE)
    re_id = re.compile("id: '([a-zA-Z0-9]*)' ?", re.IGNORECASE)
    
    encparam = re_enc.search(html).group(1)
    encid = re_id.search(html).group(1)
    
    return encparam, encid

get_basic_key_values('052860') # 종목코드 입력

('NGZRN0w2dnBYR0ZCOXFTdHlickFodz09', 'TDdCYnFkT0')

## get_basic_key_values 함수를 사용해서 Financial Summary 값을 데이터 프레임으로 불러오기

In [42]:
def get_dataframe(code):
    
    encparam, encid = get_basic_key_values(code)

    url = f'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd={code}&fin_type=0&freq_type=A&encparam={encparam}&id={encid}'
    headers = {'Referer': 'HACK'}
    html = requests.get(url, headers=headers).text
    dfs = pd.read_html(html)
    df = dfs[1]

    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ 데이터프레임 만지는 과정
    df = df.transpose() # 보기 편하게 데이터프레임의 행과 열을 바꾼다.

    find_url = f'https://finance.naver.com/item/main.naver?code={code}'

    response = requests.get(find_url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')

    # 기업명 가져오기
    name_tag = soup.find('div', {'class': 'wrap_company'}).find('h2')
    stock_name = name_tag.text.strip()


    df['기업명'] = [stock_name, f'{stock_name}(2020/12)', f'{stock_name}(2021/12)', f'{stock_name}(2022/12)', f'{stock_name}(2023/12)', f'{stock_name}(2024/12)',
                  f'{stock_name}(2025/12)(예측)', f'{stock_name}(2026/12)(예측)', f'{stock_name}(2027/12)(예측)'] # 기업명 열을 추가한다.

    # '기업명' 열을 첫 번째 열로 이동시킨다.
    # 열 순서를 직접 지정한다.
    cols = ['기업명'] + [col for col in df.columns if col != '기업명']
    df = df[cols]

    # 첫 번째 행을 열 이름으로 설정하고 나머지만 남기기
    df.columns = df.iloc[0] # 첫 행을 column으로 지정한다.
    df = df[1:].reset_index(drop=True) # 첫 행을 삭제하고 인덱스를 초기화한다.
    df = df.rename(columns={df.columns[0]: '기업명'}) # 첫 번째 열 이름을 '기업명'으로 변경


    return df

get_dataframe('052860') # 종목 코드 입력

/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


"(주요재무정보, 주요재무정보)",기업명,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
0,아이앤씨(2020/12),229.0,-56.0,-56.0,-60.0,-60.0,-61.0,1.0,630.0,153.0,...,32.01,472.8,-347.0,NaN,2767.0,1.55,0.0,0.0,0.0,17694054.0
1,아이앤씨(2021/12),299.0,-35.0,-35.0,-54.0,-54.0,-55.0,0.0,649.0,220.0,...,51.08,417.01,-307.0,NaN,2465.0,1.61,NaN,NaN,0.0,17862854.0
2,아이앤씨(2022/12),394.0,24.0,24.0,10.0,10.0,10.0,1.0,654.0,211.0,...,47.55,430.95,54.0,46.27,2537.0,0.98,NaN,NaN,0.0,17862854.0
3,아이앤씨(2023/12),643.0,14.0,14.0,1.0,0.0,-3.0,3.0,673.0,227.0,...,51.11,427.91,-17.0,NaN,2529.0,1.09,NaN,NaN,0.0,17862854.0
4,아이앤씨(2024/12),234.0,-107.0,-107.0,-120.0,-120.0,-119.0,-1.0,513.0,208.0,...,68.27,297.66,-668.0,NaN,1836.0,0.98,NaN,NaN,NaN,17862854.0
5,아이앤씨(2025/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,아이앤씨(2026/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,아이앤씨(2027/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 특정 섹터에 있는 종목들의 종목 코드 크롤링

In [43]:
# 예시 URL (업종 리스트 페이지로 대체 가능)
url = 'https://finance.naver.com/sise/sise_group_detail.naver?type=upjong&no=278'  # 반도체와반도체장비 업종

# 요청 보내기 (User-Agent 포함)
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 종목 코드 추출
stock_codes = []

for tag in soup.select('td.name a'):
    href = tag.get('href')
    if 'code=' in href:
        code = href.split('code=')[-1]
        stock_codes.append(code)

# 결과 출력
print(stock_codes)

['413300', '052860', '323350', '093640', '418420', '011930', '227950', '024850', '306620', '105330', '355150', '389020', '041520', '399720', '119830', '440110', '052900', '348350', '412350', '064290', '142210', '059090', '094170', '424980', '402490', '032580', '147760', '452430', '140070', '092220', '445090', '429270', '054450', '160980', '080580', '025560', '053610', '420770', '077500', '006200', '217190', '417840', '253590', '054940', '083310', '096610', '222160', '303030', '317330', '452160', '311320', '087600', '089890', '061970', '122640', '149010', '432720', '200710', '144960', '033170', '382800', '029460', '232680', '080220', '170920', '241790', '356860', '117670', '183300', '330860', '172670', '217500', '396470', '405100', '098120', '033160', '059120', '417500', '093520', '033640', '080520', '005935', '102120', '394280', '078350', '094360', '171010', '066310', '482630', '254490', '097800', '200470', '036200', '083450', '240810', '036540', '045300', '365590', '396270', '241770',

### 데이터프레임 합치기 (크롤링한 데이터를 하나의 데이터프레임으로 결합)

In [44]:
# 종목 코드 리스트를 사용하여 데이터프레임 생성
final_df = pd.DataFrame() # 빈 데이터프레임 생성

for code in stock_codes:
    try:
        df = get_dataframe(code) # 각 종목 코드에 대해 데이터프레임 생성
        final_df = pd.concat([final_df, df], axis=0) # 데이터프레임 합치기
    except Exception as e:
        print(f"Error processing code {code}: {e}")

final_df.reset_index(drop=True, inplace=True) # 인덱스 초기화
final_df # 최종 데이터프레임 출력

Error processing code 413300: 'NoneType' object has no attribute 'group'


/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed

Error processing code 149010: 'NoneType' object has no attribute 'group'


/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed

Error processing code 169670: 'NoneType' object has no attribute 'group'


/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_24539/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed

Error processing code 136660: 'NoneType' object has no attribute 'group'


"(주요재무정보, 주요재무정보)",기업명,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
0,아이앤씨(2020/12),229.0,-56.0,-56.0,-60.0,-60.0,-61.0,1.0,630.0,153.0,...,32.01,472.8,-347.0,NaN,2767.0,1.55,0.0,0.0,0.0,17694054.0
1,아이앤씨(2021/12),299.0,-35.0,-35.0,-54.0,-54.0,-55.0,0.0,649.0,220.0,...,51.08,417.01,-307.0,NaN,2465.0,1.61,NaN,NaN,0.0,17862854.0
2,아이앤씨(2022/12),394.0,24.0,24.0,10.0,10.0,10.0,1.0,654.0,211.0,...,47.55,430.95,54.0,46.27,2537.0,0.98,NaN,NaN,0.0,17862854.0
3,아이앤씨(2023/12),643.0,14.0,14.0,1.0,0.0,-3.0,3.0,673.0,227.0,...,51.11,427.91,-17.0,NaN,2529.0,1.09,NaN,NaN,0.0,17862854.0
4,아이앤씨(2024/12),234.0,-107.0,-107.0,-120.0,-120.0,-119.0,-1.0,513.0,208.0,...,68.27,297.66,-668.0,NaN,1836.0,0.98,NaN,NaN,NaN,17862854.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,유진테크(2023/12),2765.0,243.0,243.0,318.0,254.0,244.0,10.0,4370.0,685.0,...,18.59,2946.59,1066.0,38.26,15517.0,2.63,200.0,0.49,18.19,22916042.0
1268,유진테크(2024/12),3381.0,612.0,612.0,769.0,654.0,633.0,22.0,5395.0,1038.0,...,23.81,3455.05,2761.0,11.34,18261.0,1.71,230.0,0.73,8.09,22916042.0
1269,유진테크(2025/12)(예측),4580.0,877.0,NaN,1005.0,774.0,752.0,NaN,6230.0,1285.0,...,25.99,NaN,3284.0,10.49,20947.0,1.64,200.0,0.58,6.09,NaN
1270,유진테크(2026/12)(예측),5085.0,1058.0,NaN,1188.0,920.0,892.0,NaN,6885.0,1080.0,...,18.6,NaN,3890.0,8.86,24746.0,1.39,200.0,0.58,5.14,NaN


### 최종 데이터프레임을 csv파일로 변환

In [45]:
final_df.to_csv('financial_summary.csv', index=False, encoding='utf-8-sig')